In [ ]:
import sys
from pygame.locals import *
from utils import *

FPS = 20  # frames per second setting

fpsClock = pg.time.Clock()

pg.init()  # inicializa pygame

window = pg.display.set_mode((windowWidth, windowHeight))  # largura e altura da tela
pg.display.set_caption('Catch the ball!')
rct = pg.Rect(rctLeft, rctTop, rctWidth, rctHeight)

action = 1  # 0 faz nada, 1 esquerda, 2 direita

score = 0
missed = 0
reward = 0
font = pg.font.Font(None, 30)

In [ ]:
def windowsControl(s, act, crclCentreX, crclCentreY, pg, reward, window, score, missed):
    rct = new_rect_after_action(s.rect, act)  # nova posição da pá após ação
    crclCentreX = s.circle.circleX  # put the ball where it originally was before the experiment
    crclCentreY = s.circle.circleY

    pg.draw.circle(window, RED, (int(crclCentreX), int(crclCentreY)),int(crclRadius))
    pg.draw.rect(window, GREEN, rct)
    
    if reward == 1:  # got it!
        score += reward  # add the reward to the total score
    elif reward == -1:  # missed
        missed += reward  # add the reward to the missed count

    return score, missed 


### Q-learning

In [ ]:
def qLearning(rct, crclCentreX, crclCentreY):
    s = State(rct, Circle(crclCentreX, crclCentreY))
    act = get_best_action(s)  # pega a melhor ação do estado atual
    r0 = calculate_score(s.rect, s.circle)  # obtém a recompensa por ter feito essa ação
    s1 = new_state_after_action(s, act)  # novo estado após ter feito a ação
    # cria a tabela Q, indexada por estado e ação
    Q[state_to_number(s), act] += lr * (r0 + y * np.max(Q[state_to_number(s1), :]) - Q[state_to_number(s), act])
    
    return s, act, r0, s1, Q

### Mudar parâmetros

In [ ]:
# taxa de aprendizagem (alpha) 
#α = 0 : fará com que o agente não aprenda nada
#α = 1 : o agente consideraria apenas a informação mais recentes
lr = 0


# fator de desconto (gamma)
#γ = 0: confiaremos apenas nas recompensas imediatas.
#γ = 1 : considera o ambiente determinista e as mesmas ações sempre resultarão nas mesmas recompensas.
y = 0

In [ ]:

i = 0

while True:
    for event in pg.event.get():
        if event.type == QUIT:  # botão de fechar a janela
            pg.quit()
            sys.exit()

    window.fill(BLACK)  # cor do fundo da janela
    
    
    #'Controla' o jogo
    if crclCentreY >= windowHeight - rctHeight - crclRadius:  # checa se a pá (retângulo) está abaixo da bolinha(circulo) ou não
        reward = calculate_score(rct, Circle(crclCentreX, crclCentreY))  # +1 ou -1 calcula se perdeu ou ganhou
        crclCentreX = circle_falling(crclRadius)
        crclCentreY = 50 #recomeça
    else:
        reward = 0  # sem recompensa se a bola não estiver abaixo da pá
        crclCentreY += crclYStepFalling  # continua deixando a bola cair
        

    ### Q-learning    
    s, act, r0, s1, Q = qLearning(rct, crclCentreX, crclCentreY)    
    
    
    ### Criação e atualização da tela 
    score, missed = windowsControl(s, act, crclCentreX, crclCentreY, pg, reward, window, score, missed)
    text = font.render('score: ' + str(score), True, (238, 58, 140))  # atualiza a pontuação na tela
    text1 = font.render('missed: ' + str(missed), True, (238, 58, 140))  # atualiza a pontuação na tela
    window.blit(text, (windowWidth - 120, 10))  # renderiza o score
    window.blit(text1, (windowWidth - 280, 10))  # renderiza o missed

    pg.display.update()  # atualiza do display
    fpsClock.tick(FPS) 
    
    if i == 10000:  # condição de parada
        break
    else:
        i += 1
